In [1]:
import pandas as pd
import numpy as np
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder, Imputer

In [3]:
train_df = pd.read_csv('/var/data/train.csv')
test_df = pd.read_csv('/var/data/test.csv')

Use IDs as index

In [4]:
train_df.set_index('Id', inplace=True, verify_integrity=True)
test_df.set_index('Id', inplace=True, verify_integrity=True)

### Cleaning

The target variable has to be called **class** (tpot internal constraint)

In [5]:
train_df.rename(columns={'SalePrice': 'class'}, inplace=True)

#### Missing values imputation

In [6]:
train_df.LotFrontage.fillna(value=0.0, inplace=True)
train_df.Alley.fillna(value='No alley', inplace=True)
train_df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']] = train_df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna(value='No basement')
train_df.FireplaceQu.fillna(value='No fireplace', inplace=True)
train_df[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']] = train_df[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']].fillna(value='No garage')
train_df.PoolQC.fillna(value='No pool', inplace=True)
train_df.Fence.fillna(value='No fence', inplace=True)
train_df.MiscFeature.fillna(value='No extra', inplace=True)

train_df.MasVnrType.fillna(value='Unknown', inplace=True)
train_df.drop(['MasVnrArea', 'GarageYrBlt', 'MiscVal'], axis=1, inplace=True)

Fill missing value missing value for variable _Electrical_ with a possible value

In [8]:
print(train_df.loc[train_df.Electrical.isnull(), 'Electrical'])

Id
1380    NaN
Name: Electrical, dtype: object


In [9]:
train_df.Electrical.describe()

count      1459
unique        5
top       SBrkr
freq       1334
Name: Electrical, dtype: object

In [10]:
train_df.Electrical.fillna(value='SBrkr', inplace=True)

In [9]:
# # train_df.drop(train_df.loc[train_df.Electrical.isnull()].index, inplace=True)
# imputer = Imputer(strategy='most_frequent', axis=0, copy=True)
# impute_model = imputer.fit(train_df)
# train_df_ = impute_model.transform(train_df)
# print train_df_.loc[train_df.Electrical.isnull(), 'Electrical']

### Transform categorical variables into numerical

#### Label Binariser
Transform categorical variables into one-hot-encoded variables

In [11]:
lb_var_list = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 
               'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
               'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 
               'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
               'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
               'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
lb_transfo_list = ['lb_' + var for var in lb_var_list]
lb_model_list = ['lb_' + var + '_model' for var in lb_var_list]
lb_train_list = [var + '_train' for var in lb_var_list]

lb_transfo_dict = {}
for transfo in lb_transfo_list:
    lb_transfo_dict[transfo] = LabelBinarizer()

lb_model_dict = {}
lb_train_dict = {}
for i in range(len(lb_var_list)):
    lb_model_dict[lb_model_list[i]] = lb_transfo_dict[lb_transfo_list[i]].fit(train_df[lb_var_list[i]])
    lb_train_dict[lb_train_list[i]] = lb_model_dict[lb_model_list[i]].transform(train_df[lb_var_list[i]])

In [12]:
len(lb_train_dict)

43

In [13]:
lb_train_dict['MSZoning_train'].shape

(1460, 5)

#### One Hot Encoder
Transform numerical variables into one-hot-encoded variables

In [14]:
ohe_var_list = ['MSSubClass', 'MoSold']
ohe_transfo_list = ['ohe_' + var for var in ohe_var_list]
ohe_model_list = ['ohe_' + var + '_model' for var in ohe_var_list]
ohe_train_list = [var + '_train' for var in ohe_var_list]

ohe_transfo_dict = {}
for transfo in ohe_transfo_list:
    ohe_transfo_dict[transfo] = OneHotEncoder(sparse=False)

ohe_model_dict = {}
ohe_train_dict = {}
for i in range(len(ohe_var_list)):
    ohe_model_dict[ohe_model_list[i]] = ohe_transfo_dict[ohe_transfo_list[i]].fit(train_df[ohe_var_list[i]].values.reshape(-1, 1))
    ohe_train_dict[ohe_train_list[i]] = ohe_model_dict[ohe_model_list[i]].transform(train_df[ohe_var_list[i]].values.reshape(-1, 1))

In [15]:
len(ohe_train_dict)

2

In [16]:
ohe_train_dict['MSSubClass_train'].shape

(1460, 15)

In [17]:
ohe_train_dict['MoSold_train'].shape

(1460, 12)

#### Concatenate transformed features

In [18]:
# Untouched variables
unt_var_list = list(set(train_df.columns) - set(lb_var_list) - set(ohe_var_list) - {'class'})

In [36]:
tuple([value[1] for value in lb_train_dict.items()])

(array([[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        ..., 
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0]]), array([[1],
        [1],
        [1],
        ..., 
        [1],
        [1],
        [1]]), array([[0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        ..., 
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]]), array([[0, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        ..., 
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1]]), array([[0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        ..., 
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1]]), array([[0],
        [0],
        [0],
        ..., 
        [0],
        [0],
        [0]]), array([[0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1],
        ..., 
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1]]), array([[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
    

In [37]:
array_unt = train_df.as_matrix(columns=unt_var_list)
array_lb = np.hstack(tuple([value[1] for value in lb_train_dict.items()]))
array_ohe = np.hstack(tuple([value[1] for value in ohe_train_dict.items()]))

In [38]:
train_array = np.hstack((array_unt, array_lb, array_ohe))

In [39]:
train_array.shape

(1460, 322)

### tpot

In [41]:
X_train, X_test, y_train, y_test = train_test_split(train_array, train_df['class'], train_size=0.95, test_size=0.05)

tpot = TPOTRegressor(generations=100, population_size=10, verbosity=2, n_jobs=-1, random_state=26)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

# About the warning
# https://stackoverflow.com/questions/41238769/warning-messages-when-using-python
# https://github.com/rhiever/tpot/issues/284

Optimization Progress:   2%|▏         | 20/1010 [00:41<3:10:30, 11.55s/pipeline]

Generation 1 - Current best internal CV score: 904943648.044514


Optimization Progress:   3%|▎         | 30/1010 [01:47<4:12:41, 15.47s/pipeline]

Generation 2 - Current best internal CV score: 904943648.044514


Optimization Progress:   4%|▍         | 40/1010 [01:57<2:27:07,  9.10s/pipeline]

...

Optimization Progress: 100%|█████████▉| 1006/1010 [23:27:37<2:08:54, 1933.57s/pipeline]

Generation 98 - Current best internal CV score: 527850570.6127537


Optimization Progress: 1016pipeline [23:28:13, 951.21s/pipeline]                       

Generation 99 - Current best internal CV score: 527850570.6127537


Generation 100 - Current best internal CV score: 527850570.6127537

Best pipeline: XGBRegressor(LinearSVR(CombineDFs(LinearSVR(CombineDFs(input_matrix, input_matrix), C=0.1, dual=False, epsilon=0.1, loss=squared_epsilon_insensitive, tol=1e-05), MaxAbsScaler(input_matrix)), C=0.1, dual=False, epsilon=0.1, loss=squared_epsilon_insensitive, tol=0.1), learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=100, nthread=1, subsample=0.95)
487708836.924


In [42]:
tpot.export('tpot_exported_pipeline.py')

True

In [43]:
tpot.evaluated_individuals_

{'DecisionTreeRegressor(DecisionTreeRegressor(input_matrix, DecisionTreeRegressor__max_depth=3, DecisionTreeRegressor__min_samples_leaf=9, DecisionTreeRegressor__min_samples_split=13), DecisionTreeRegressor__max_depth=9, DecisionTreeRegressor__min_samples_leaf=9, DecisionTreeRegressor__min_samples_split=7)': (2,
  -1656784604.539309),
 'DecisionTreeRegressor(LinearSVR(input_matrix, LinearSVR__C=0.1, LinearSVR__dual=True, LinearSVR__epsilon=0.001, LinearSVR__loss=squared_epsilon_insensitive, LinearSVR__tol=0.01), DecisionTreeRegressor__max_depth=6, DecisionTreeRegressor__min_samples_leaf=4, DecisionTreeRegressor__min_samples_split=11)': (2,
  -1206903849.5031352),
 'DecisionTreeRegressor(LinearSVR(input_matrix, LinearSVR__C=0.5, LinearSVR__dual=True, LinearSVR__epsilon=0.001, LinearSVR__loss=squared_epsilon_insensitive, LinearSVR__tol=1e-05), DecisionTreeRegressor__max_depth=10, DecisionTreeRegressor__min_samples_leaf=16, DecisionTreeRegressor__min_samples_split=3)': (2,
  -1216302454.1